## 1. Introduction

This report validates the accuracy of KS test and bootstrapping Sharpe by simulating market alpha series over time. All simulated alpha series follow the same distribution and hence both tests should be able to pick number of inconsistent tests according to significant levels.

## 2.Data

_BWD_Alpha-MCWTR-BrBetaR3KFast_C2C_W1.csv

'2010-01-01':'2018-12-31'

In [190]:
%matplotlib inline

import validation
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
import pyflux as pf
import warnings
from scipy import stats

warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = [15,8]

In [191]:
dir_pooldump= r"\\farmnas\farm2\Research\DH_FUNDAMENTAL\Data\PHRES-608\20190802\_poolDump\\"

In [192]:
def merge_partitions(partitions, list_num_par):

    '''

    merge dataframe partitions

    Args:

        partitions (dict): dict of dataframes
        list_num_par (list of int): list of dataframe indexes to merge

    Returns:

        df (pandas.DataFrame): merged df

    '''

    df = partitions[0]

    for num_par in list_num_par[1:]:

        df=df.append(partitions[num_par])

    return df

def get_CI(series, ci):
    
    """
    calculate lower and upper bound at confidence level ci
            
    """
    
    lower_bound=np.percentile(series,  (100-ci)/2)
    upper_bound=np.percentile(series,  ci+(100-ci)/2)
    
    return lower_bound, upper_bound


## 3. Method

EW daily alpha across pool is calculated and used as a continue alpha series for resampling

### Boostrapping Daily EW Alpha Series  

In [193]:
num_simulation = 10000
df_pool = pd.read_csv(dir_pooldump + "_PI.csv", index_col = 0)
df_1D_Alpha = pd.read_csv(dir_pooldump + "_BWD_Alpha-MCWTR-BrBetaR3KFast_C2C_W1.csv", index_col = 0).ix['2010-01-01':'2018-12-31']
df_1D_Alpha = df_1D_Alpha[df_pool == 1]
df_1D_market_alpha = df_1D_Alpha.mean(axis = 1)

partitions=np.array_split(df_1D_market_alpha, num_simulation)

list_partitions=list(range(num_simulation))

df_simulaiton = pd.DataFrame(index = df_1D_market_alpha.index, columns = list_partitions)

for i in range(num_simulation):

    df_simulaiton[i] = np.random.choice(df_1D_market_alpha.fillna(0).tolist(), len(df_1D_market_alpha), replace = True)

### KS Test

In [194]:
df_ks = pd.DataFrame(index = range(num_simulation), columns = ['ks_pvalue'])

for i in range(num_simulation):
        
    train_ret = df_simulaiton[i].loc['2010-01-01':'2015-12-31']
    valid_ret =  df_simulaiton[i].loc['2016-01-01':'2018-12-31']

    rvs1 = train_ret.values
    rvs2 = valid_ret.values
    
    _, _pValue = stats.ks_2samp(rvs1, rvs2)
    df_ks.loc[i] = float(_pValue)

### 510 simulations out of 10000 have p-value less than 5% 

In [195]:
df_ks = df_ks.astype(float)
df_ks.describe()

,ks_pvalue
count,10000.000000
mean,0.511600
std,0.295090
min,0.000172
25%,0.260344
50%,0.516758
75%,0.771727
max,0.999830


In [196]:
(df_ks < 0.05).sum()

ks_pvalue    510
dtype: int64

### Sharpe Boostrapping

In [197]:
df_test = pd.DataFrame(df_1D_market_alpha)

In [198]:
partition_size = 5

perf_metric_func = lambda df: df.mean(axis=0) / df.std(axis=0)*(252**0.5)

df_boostrap_sharpe_val = pd.DataFrame(index=range(num_simulation), columns=['lower_bound_95', 'upper_bound_95',
                                                                            'lower_bound_90', 'upper_bound_90',
                                                                            'observed_value'])

for _simu in range(num_simulation):

    train_series = df_simulaiton[_simu].loc['2010-01-01':'2015-12-31']
    valid_series = df_simulaiton[_simu].loc['2016-01-01':'2018-12-31']

    train_num_partition=int(train_series.shape[0] / partition_size)
    valid_num_partition=int(valid_series.shape[0] / partition_size)

    partitions=np.array_split(train_series, train_num_partition)

    list_partitions=list(range(train_num_partition))

    df_sharpe_simulaiton=pd.DataFrame(index=range(100), columns=['measurement'])

    for i in range(100):

        simulated_valid_partition_num=np.random.choice(list_partitions, valid_num_partition, replace=True)

        df_simulated_valid=merge_partitions(partitions, simulated_valid_partition_num)

        df_sharpe_simulaiton.ix[i]=perf_metric_func(df_simulated_valid)

    lower_bound_95, upper_bound_95 = get_CI(df_sharpe_simulaiton['measurement'], 95)
    lower_bound_90, upper_bound_90 = get_CI(df_sharpe_simulaiton['measurement'], 90)
    observed_value = perf_metric_func(valid_series)
    
    temp = [lower_bound_95, upper_bound_95, lower_bound_90, upper_bound_90, observed_value]
    df_boostrap_sharpe_val.ix[_simu] = temp

### the test rejects 1401 out of 10000 series with 95% CI and rejects 2081 out of 10000 series with 90% CI

In [201]:
df_boostrap_sharpe_val[(df_boostrap_sharpe_val['observed_value'] < df_boostrap_sharpe_val['lower_bound_95']) 
                      | (df_boostrap_sharpe_val['observed_value'] > df_boostrap_sharpe_val['upper_bound_95'])].shape[0]

1401

In [204]:
df_boostrap_sharpe_val[(df_boostrap_sharpe_val['observed_value'] < df_boostrap_sharpe_val['lower_bound_90']) 
                      |(df_boostrap_sharpe_val['observed_value'] > df_boostrap_sharpe_val['upper_bound_90'])].shape[0]

2081

## 4. Result

KS is more accurate in this test and boostrapping Sharpe seems to have higher type 1 error than expected

## 5. Conclusion

5% p-value in KS test has been selected as cutoff point to validate strategy's performance and boostrapping sharpe is supplimentary test. For strategies that do not pass the tests, discretionary decision can be made to take the strategy to next step.